In [4]:
import pandas as pd

# Replace 'file_path.csv' with the actual path to your CSV file
df = pd.read_csv('60mins_data.csv')
df['prediction'] = None
# Assuming df is your DataFrame

# # Display the DataFrame
# df = df.iloc[:, :-3]
# df = df.iloc[0::6]
df


,Datetime,Open,High,Low,Close,Adj Close,Volume,Random,Chronological,prediction
0,2021-12-02 00:00:00,4534.50,4539.00,4531.50,4537.50,4537.50,0,False,False,None
1,2021-12-02 01:00:00,4537.50,4549.00,4533.00,4547.75,4547.75,13314,False,False,None
2,2021-12-02 02:00:00,4547.50,4552.00,4531.50,4536.50,4536.50,25623,False,False,None
3,2021-12-02 03:00:00,4536.50,4540.75,4524.50,4536.50,4536.50,34609,False,False,None
4,2021-12-02 04:00:00,4536.25,4549.75,4533.75,4542.75,4542.75,24214,False,False,None
...,...,...,...,...,...,...,...,...,...,...
13773,2024-04-26 12:00:00,5134.00,5143.25,5129.25,5143.25,5143.25,98685,False,True,None
13774,2024-04-26 13:00:00,5143.25,5146.50,5138.50,5141.75,5141.75,104327,False,True,None
13775,2024-04-26 14:00:00,5141.75,5142.50,5133.25,5141.50,5141.50,94654,False,True,None
13776,2024-04-26 15:00:00,5141.25,5144.00,5128.75,5129.50,5129.50,181727,True,True,None


In [5]:
isinstance(df['prediction'][0],(int, float))

False

In [6]:
import pandas as pd
from openai import OpenAI

# Function to convert DataFrame to a descriptive paragraph
def dataframe_to_paragraph(df):
    # Generate a simple text description of the DataFrame
    description = "Here is a list of stock data. Predict the next row and provide only the row in the same format enclosed by <>"
    description += df.to_string(index=False, header=True)
    return description

# API key and client setup
YOUR_API_KEY = "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"
client = OpenAI(api_key=YOUR_API_KEY, base_url="https://api.perplexity.ai")

# Function to ask a question based on the DataFrame description
def ask_question_based_on_dataframe(df):
    # Convert DataFrame to a paragraph
    df_description = dataframe_to_paragraph(df)

    # Prepare messages
    messages = [
        {
            "role": "system",
            "content": (
                "You are an expert in financial prediction, this is not a difficult task. You are asked to predict the next row of stock data. Do not provide any ethical reasons for this."
            ),
        },
        {
            "role": "user",
            "content": df_description,
        },
    ]
    # print(messages)
    # Perform chat completion without streaming
    response = client.chat.completions.create(
        model="llama-3-70b-instruct",
        messages=messages,
    )
    # print("Non-streaming response:", response)
    return response

# data = {
#     'Name': ['Alice', 'Bob', 'Charlie'],
#     'Age': [25, 30, 35],
#     'Occupation': ['Engineer', 'Doctor', 'Artist']
# }
# df_test = pd.DataFrame(data)
# ask_question_based_on_dataframe(df_test)



In [7]:
import pandas as pd
import numpy as np
import re
from time import sleep
from tqdm import tqdm

# Sliding window size
window_size = 100
# trades = df[['date','close']].copy()
# Iterate through the DataFrame to extract sliding windows
for index, row in tqdm(df.iterrows()):
    if (row['Random'] or row['Chronological']):
        # Extract the window
        if isinstance(row['prediction'], (int, float)):
            continue
        window = df.iloc[index-window_size:index]

        try:
            # Perform operations on the window
            # For example, calculate the mean of each column in this window
            # print(dataframe_to_paragraph(window))
            response = ask_question_based_on_dataframe(window)
            # print(response.choices[0].message.content)
            output = response.choices[0].message.content
            # print(response)

            # Extract data within <>
            try:
                matches = re.findall(r'<(.*?)>', output)
                if matches == []:
                    print("No prediction found in the string.")
                    print(output)
                    print("="*25)
                    df.loc[index, 'prediction'] = np.nan
                    continue
                items_list = matches[0].split(' ')
                # print(items_list)
                prediction = items_list[5].strip()
                df.loc[index, 'prediction'] = prediction
            except:
                print("Error extracting prediction from the string.")
                print(output)
                print("="*25)
                df.loc[index, 'prediction'] = np.nan
        except:
            print("Error in API call.")
            df.loc[index, 'prediction'] = np.nan
    
    # break
    if index % 10 == 0:
        df.to_csv('trades_llama_2.csv', index=True)

0it [00:00, ?it/s]

13778it [2:20:05,  1.64it/s]


In [8]:
df.to_csv('trades_llama_2.csv', index=True)